In [1]:
from datasets import load_dataset

ds = load_dataset("saiyan-world/Goku-MovieGenBench")

/Users/salmanahmadgill/LLM-Model/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['video'],
        num_rows: 1003
    })
})


In [3]:
print(ds['train'].features)


{'video': Video(decode=True, id=None)}


In [4]:
video_path = ds['train'][0]['video']
print(f"Video Path: {video_path}")

Video Path: <decord.video_reader.VideoReader object at 0x14ee3f310>


In [5]:
import cv2
import numpy as np

video_reader = ds['train'][3]['video']

for i in range(len(video_reader)):
    frame = video_reader[i].asnumpy()  

    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    cv2.imshow('Video Frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()



2025-02-18 03:49:25.800 python[50900:2552086] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-18 03:49:25.800 python[50900:2552086] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [6]:
from transformers import AutoModelForVideoClassification, VideoMAEImageProcessor
import torch

model = AutoModelForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")
processor = VideoMAEImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400")

def preprocess_video(video):
    frames = []
    for i in range(len(video)):
        frame = video[i].asnumpy()
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frames.append(frame)
    return np.array(frames)

train_videos = [preprocess_video(video['video']) for video in ds['train']]

inputs = processor(train_videos, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

# Print logits
print(outputs.logits)

: 